In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from VirtualWorld.World import World
from Knowledge.Article import Article
from Knowledge.Source import Source

In [3]:
world = World()

Big Bang!
Tanja Akin Bias.CENTER
Beatrice Hall Bias.CENTER
Anthony Navarro Bias.CENTER
William Washinton Bias.CENTER
Bruce Christensen Bias.CENTER
Peggy Gore Bias.CENTER
Connie Perry Bias.CENTER
William Smyth Bias.CENTER
Lauren Allen Bias.CENTER
Michael Coates Bias.CENTER
Maria Helms Bias.CENTER
Ann Meyer Bias.CENTER
Debra Rio Bias.CENTER
Jessie Mason Bias.CENTER
Lisa Thompson Bias.CENTER
Marion Garza Bias.CENTER
Virginia Mcclure Bias.CENTER
Janice Polley Bias.CENTER
Blanche Giambra Bias.CENTER
James Sharma Bias.CENTER
Brandon Bergman Bias.LEAN_RIGHT
Jean Sabin Bias.CENTER
Gloria Manion Bias.CENTER
Kaitlin Harris Bias.CENTER
Dale Frazier Bias.CENTER
Mary Morvant Bias.CENTER
Samuel Leversee Bias.CENTER
Kevin Allen Bias.CENTER
William Clark Bias.CENTER
Frances Strahl Bias.CENTER
Debra Mcdonald Bias.CENTER
Jane Quan Bias.CENTER
Nancy Zuniga Bias.CENTER
Matthew Ennenga Bias.CENTER
Francis Tu Bias.CENTER
Janet Jones Bias.CENTER
Tyler Golden Bias.CENTER
Alan Rodriguez Bias.CENTER
Mary Sheldo

In [ ]:
topic = world.topics[63]
world.knowledge.filter_articles_by_topic(topic)

In [ ]:
from NPC.Person import Person

In [ ]:
Person.living_population

In [ ]:
world.towns['Raleigh'].citizens[10].house_number

In [ ]:
world.towns['Raleigh'].citizens[10].name

In [ ]:
world.population

In [ ]:
for house in world.towns['Raleigh'].homes.values(): 
    print(house['family'], house['occupants'])